# New 02 - Task 02 - VetCyto - MOBILENETv3L - Testing Code (Manually Labeled)


### alternative code 01 (for making predictions in only a single target folder containing images)

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
import matplotlib.pyplot as plt

def load_trained_model(checkpoint_path):
    model = tf.keras.models.load_model(checkpoint_path)
    return model

def prepare_image(img_path, target_size):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

def batch_predict(model, img_arrays):
    img_batch = np.vstack(img_arrays)
    predictions = model.predict(img_batch)
    return predictions

def predict_images_in_folder(model, folder_path, class_indices, batch_size=32):
    img_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    img_arrays = []
    predictions = {}
    class_counts = {class_name: 0 for class_name in class_indices.keys()}
    
    for img_file in img_files:
        img_path = os.path.join(folder_path, img_file)
        img_array = prepare_image(img_path, target_size=(128, 128))
        img_arrays.append(img_array)
        
        if len(img_arrays) == batch_size:
            batch_predictions = batch_predict(model, img_arrays)
            for i, pred in enumerate(batch_predictions):
                predicted_class = np.argmax(pred)
                class_name = list(class_indices.keys())[predicted_class]
                predictions[img_files[i]] = class_name
                class_counts[class_name] += 1
            img_arrays = []

    if img_arrays:
        batch_predictions = batch_predict(model, img_arrays)
        for i, pred in enumerate(batch_predictions):
            predicted_class = np.argmax(pred)
            class_name = list(class_indices.keys())[predicted_class]
            predictions[img_files[-len(img_arrays) + i]] = class_name
            class_counts[class_name] += 1

    total_images = len(img_files)
    class_percentages = {class_name: (count / total_images) * 100 for class_name, count in class_counts.items()}
    
    return predictions, class_counts, class_percentages

# default = 'C:/Users/karli/Desktop/outputMobile/MobileNet_87_97.keras' ; alt = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\new_work_01\\outputMOBILENETv3L\\MobileNet_50_81.keras' ;
checkpoint_path = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\new_work_02\\outputMOBILENETv3L\\MobileNet_58_93.keras' ;
# default = 'C:/Users/karli/Desktop/vet_images_sliced_split(training)/test/9' ; alt = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\new_work_01\\vet_images_sliced\\TestingStep\\TE(10)TAPE' ;
folder_path = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\new_work_02\\vet_images_sliced_split\\TrainingStepSet\\test\\T2' ;

model = load_trained_model(checkpoint_path)

class_indices = {'T1': 0, 'T2': 1}

predictions, class_counts, class_percentages = predict_images_in_folder(model, folder_path, class_indices, batch_size=32)


# Print predictions
for img_file, class_name in predictions.items():
    print(f"{img_file}: {class_name}")

# Print class counts and percentages
print("\nClass Counts:")
for class_name, count in class_counts.items():
    print(f"{class_name}: {count}")

print("\nClass Percentages:")
for class_name, percentage in class_percentages.items():
    print(f"{class_name}: {percentage:.2f}%")


### alternative code 02 (for maing predictions in multiple target folders with a shared parent folder)

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
import matplotlib.pyplot as plt

def load_trained_model(checkpoint_path):
    """Load the trained model from the checkpoint path."""
    model = tf.keras.models.load_model(checkpoint_path)
    return model

def prepare_image(img_path, target_size):
    """Preprocess the image to the required input format for the model."""
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

def batch_predict(model, img_arrays):
    """Predict the batch of images using the trained model."""
    img_batch = np.vstack(img_arrays)
    predictions = model.predict(img_batch)
    return predictions

def predict_images_in_folder(model, folder_path, class_indices, batch_size=32):
    """Predict all images in a given folder and return the predictions, class counts, and class percentages."""
    img_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    img_arrays = []
    predictions = {}
    class_counts = {class_name: 0 for class_name in class_indices.keys()}
    
    for img_file in img_files:
        img_path = os.path.join(folder_path, img_file)
        img_array = prepare_image(img_path, target_size=(128, 128))
        img_arrays.append(img_array)
        
        if len(img_arrays) == batch_size:
            batch_predictions = batch_predict(model, img_arrays)
            for i, pred in enumerate(batch_predictions):
                predicted_class = np.argmax(pred)
                class_name = list(class_indices.keys())[predicted_class]
                predictions[img_files[i]] = class_name
                class_counts[class_name] += 1
            img_arrays = []

    if img_arrays:
        batch_predictions = batch_predict(model, img_arrays)
        for i, pred in enumerate(batch_predictions):
            predicted_class = np.argmax(pred)
            class_name = list(class_indices.keys())[predicted_class]
            predictions[img_files[-len(img_arrays) + i]] = class_name
            class_counts[class_name] += 1

    total_images = len(img_files)
    class_percentages = {class_name: (count / total_images) * 100 for class_name, count in class_counts.items()}
    
    return predictions, class_counts, class_percentages, total_images

def predict_images_in_root_folder(model, root_folder_path, class_indices, batch_size=32):
    """Iterate through all subdirectories in the root folder and predict images in each subdirectory."""
    subfolders = sorted([f.path for f in os.scandir(root_folder_path) if f.is_dir()])
    
    for subfolder in subfolders:
        img_files = [f for f in os.listdir(subfolder) if os.path.isfile(os.path.join(subfolder, f))]
        total_files = len(img_files)
        print(f"\nStarting predictions for subdirectory: {subfolder}")
        print(f"Total number of files in this subdirectory: {total_files}")
        
        predictions, class_counts, class_percentages, _ = predict_images_in_folder(model, subfolder, class_indices, batch_size)
        
        # Print predictions
        print("\nPredictions:")
        for img_file, class_name in predictions.items():
            print(f"{img_file}: {class_name}")
        
        # Print class counts and percentages
        print("\nClass Counts:")
        for class_name, count in class_counts.items():
            print(f"{class_name}: {count}")
        
        print("\nClass Percentages:")
        for class_name, percentage in class_percentages.items():
            print(f"{class_name}: {percentage:.2f}%")
        
        print(f"Finished predictions for subdirectory: {subfolder}")
        print(f"Total number of files processed: {total_files}\n")

# Example usage
checkpoint_path = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\new_work_02\\outputMOBILENETv3L\\MobileNet_58_93.keras'
root_folder_path = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\new_work_02\\vet_images_sliced\\TestingStepSet'

model = load_trained_model(checkpoint_path)
class_indices = {'T1': 0, 'T2': 1}

predict_images_in_root_folder(model, root_folder_path, class_indices, batch_size=32)
